# Cell 1 - Import, percorsi, autenticazione Spotify

In [1]:
import os
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth

DATA_PROCESSED_DIR = os.path.join("..", "data", "processed")
os.makedirs(DATA_PROCESSED_DIR, exist_ok=True)

print("✓ Imports OK")

✓ Imports OK


## Cell 2 - Song's Dataset

## Kaggle set

In [2]:
kaggle_path = "/workspaces/Music_Mood_recomender/data/data.csv"

df= pd.read_csv(kaggle_path)
print("Kaggle dataset shape:", df.shape)
df.head()


Kaggle dataset shape: (169909, 19)


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [3]:
rename_cols = {
    "id": "track_id",
    "name": "track_name",
    "artists": "artist_name",
    "popularity": "popularity",
    "release_date": "release_date"
}

df = df.rename(columns=rename_cols)

# colonne da mantenere
keep = [
    "track_id", "track_name", "artist_name", "popularity", "release_date",
    "acousticness","danceability","energy","instrumentalness","liveness",
    "loudness","speechiness","tempo","valence","duration_ms"
]

df = df[keep].dropna(subset=["track_id"])
df["source_type"] = "kaggle"
df["source_label"] = "spotify_1921_2020"

print("Clean Kaggle shape:", df.shape)
df.head()


Clean Kaggle shape: (169909, 17)


,track_id,track_name,artist_name,popularity,release_date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms,source_type,source_label
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],0,1928,0.995,0.708,0.1950,0.563,0.1510,-12.428,0.0506,118.469,0.7790,158648,kaggle,spotify_1921_2020
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",0,1928,0.994,0.379,0.0135,0.901,0.0763,-28.454,0.0462,83.972,0.0767,282133,kaggle,spotify_1921_2020
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],0,1928,0.604,0.749,0.2200,0.000,0.1190,-19.924,0.9290,107.177,0.8800,104300,kaggle,spotify_1921_2020
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],0,1928-09-25,0.995,0.781,0.1300,0.887,0.1110,-14.734,0.0926,108.003,0.7200,180760,kaggle,spotify_1921_2020
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1,1928,0.990,0.210,0.2040,0.908,0.0980,-16.829,0.0424,62.149,0.0693,687733,kaggle,spotify_1921_2020


In [4]:
df = df.dropna(subset=["track_id"])
df["track_id"] = df["track_id"].astype(str).str.strip()
df = df[df["track_id"] != ""]

df = df.drop_duplicates(subset="track_id").reset_index(drop=True)

print("Shape dopo deduplica:", df.shape)

Shape dopo deduplica: (169909, 17)


In [5]:
# Convertiamo release_date in anno (utile per modello)
def extract_year(x):
    try:
        return int(str(x)[:4])
    except:
        return np.nan

df["year"] = df["release_date"].apply(extract_year)

# Riordino colonne
cols_new_order = [
    "track_id", "track_name", "artist_name", "popularity", "year",
    "acousticness", "danceability", "energy", "instrumentalness", "liveness",
    "loudness", "speechiness", "tempo", "valence", "duration_ms"
]

df = df[cols_new_order]
df.head()


,track_id,track_name,artist_name,popularity,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],0,1928,0.995,0.708,0.1950,0.563,0.1510,-12.428,0.0506,118.469,0.7790,158648
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",0,1928,0.994,0.379,0.0135,0.901,0.0763,-28.454,0.0462,83.972,0.0767,282133
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],0,1928,0.604,0.749,0.2200,0.000,0.1190,-19.924,0.9290,107.177,0.8800,104300
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],0,1928,0.995,0.781,0.1300,0.887,0.1110,-14.734,0.0926,108.003,0.7200,180760
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1,1928,0.990,0.210,0.2040,0.908,0.0980,-16.829,0.0424,62.149,0.0693,687733


In [6]:
output_path = os.path.join(DATA_PROCESSED_DIR, "spotify_dataset_clean.csv")
df.to_csv(output_path, index=False)

print("🎉 Dataset pulito salvato in:")
print(output_path)

🎉 Dataset pulito salvato in:
../data/processed/spotify_dataset_clean.csv
